# Experiments

Machine learning (ML) requires a systematic approach to understanding the data and models used to make predictions. There are a large number of variables that can change the behavior and outcome of an ML model.  The choice the data scientist makes for these variables can mean the difference between the success and failure of the model. Most often, the choices made in one situation cannot be carried over to a new model or experiment. Each new variable requires revisiting the decision of an algorithm and all the parameters associated with it. It is not possible to provide a "best choice" for a given task without more experimenting.   

The Cortex Python SDK tracks the choices the data scientist makes to improve the performance of the ML model and facilitates support of ML tasks through experiments. An `experiment` is a container for `runs`. `runs` are associated parameters, metrics, and artifacts created in the process of identifying the best algorithms for modeling a skill.

## Create an Experiment

Experiments are created using the Cortex Client:

In [4]:
from cortex import Cortex

client = Cortex.client()

exp_name = input("namespace/exp_name")
exp = client.experiment(exp_name)
exp

namespace/exp_name surface/exptest


ID,Date,Took,Params,Metrics


Executing the preceeding cell displays the experiment's `runs` as a table. The __ID__ is generated by the `run` and is a [cuid](https://github.com/ericelliott/cuid).  The __Date__ is the time for the `run` the down to the second, formatted in GMT time. Each experiment `run` is timed and the __Took__ column displays the experiment run elpase time. __Params__ and __Metrics__ are keyword arguments that you can use to configure a `run`. The empty table is populated as we create and execute `run`s.

### Experiments depend on data

Experiments run on datasets. This example uses the [UCI Iris dataset](https://archive.ics.uci.edu/ml/datasets/Iris).

In [5]:
import pandas as pd

df = pd.read_csv('./data/iris.data')

Create a training set and a test set from this data source. Use the sklearn facility to do this:

In [7]:
from sklearn.model_selection import train_test_split

all_inputs = df[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']].values
all_classes = df['Class'].values

(train_inputs, test_inputs, train_classes, test_classes) = train_test_split(all_inputs, all_classes, test_size=0.333, train_size=0.667)

## Create Runs

In this example, two runs are created for this experiment using a [decision tree classifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html). The first uses [gini impurity](https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity) as a loss funtion: 

In [8]:
from sklearn.tree import DecisionTreeClassifier

dtc_g = DecisionTreeClassifier(criterion='gini')

dtc_g_run = exp.start_run()

dtc_g_run.start()
dtc_g.fit(train_inputs, train_classes)
dtc_g_run.stop()

Run a second experiment using [information gain](https://en.wikipedia.org/wiki/Information_gain_in_decision_trees) (specified by the parameter `entropy`) for the loss function. Here the `run` context manager is used (which manages the start and stop of a run), making the code more readable.

In [9]:
dtc_e = DecisionTreeClassifier(criterion='entropy')

with exp.start_run() as run:
    dtc_e.fit(train_inputs, train_classes)

dtc_e_run = run

## Run Logging

Runs have parameters, metrics, metadata, and artifacts that can be used to track and manage experiment results.   

In [10]:
dtc_g_run.set_meta('model','DecisionTreeClassifier')
dtc_g_run.log_param('criterion','gini')
dtc_g_run.log_artifact('model',dtc_g)
dtc_g_run.log_metric('score',dtc_g.score(test_inputs, test_classes))
exp.save_run(dtc_g_run)

dtc_e_run.set_meta('model','DecisionTreeClassifier')
dtc_e_run.log_param('criterion','entropy')
dtc_e_run.log_artifact('model',dtc_e)
dtc_e_run.log_metric('score',dtc_e.score(test_inputs, test_classes))
exp.save_run(dtc_e_run)

Examine the experiment:

In [11]:
exp

The runs can also be examined:

In [12]:
dtc_g_run.to_json()

{'id': 'na7p0f4b',
 'startTime': 1562701662,
 'endTime': 1562701662,
 'took': 0.21296709999978702,
 'params': {'criterion': 'gini'},
 'metrics': {'score': 0.92},
 'meta': {'model': 'DecisionTreeClassifier'},
 'artifacts': ['model']}